In [ ]:
import numpy as np

In [ ]:
hexa = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']

In [ ]:
chi_inverse = {
        '00000':'00000',
        '00101':'00001',
        '01010':'00010',
        '01011':'00011',
        '10100':'00100',
        '10001':'00101',
        '10110':'00110',
        '10111':'00111',
        '01001':'01000',
        '01100':'01001',
        '00011':'01010',
        '00010':'01011',
        '01101':'01100',
        '01000':'01101',
        '01111':'01110',
        '01110':'01111',
        '10010':'10000',
        '10101':'10001',
        '11000':'10010',
        '11011':'10011',
        '00110':'10100',
        '00001':'10101',
        '00100':'10110',
        '00111':'10111',
        '11010':'11000',
        '11101':'11001',
        '10000':'11010',
        '10011':'11011',
        '11110':'11100',
        '11001':'11101',
        '11100':'11110',
        '11111':'11111',
        }


In [ ]:
myhash2="08060E02000000000E078E8266E964E1696E6E62000000006168606000000000"
myhash1="61686060000000000601808066E964E16F6FEEE266E964E10E078E8266E964E1"

In [ ]:
rounds = 24
#current_round=23
r=576
b=1600
c=1024

In [ ]:
def break_sha(cipher_hash,rounds): 
    k=256-4
    l= 512
    empty_array =[0]*5*5*64
    state = np.array(empty_array).reshape(5,5,64)
    c_ind =len(cipher_hash)-1 

    while k>=0:
        index = hexa.index(cipher_hash[c_ind])
        for j in range(0,4,1):
            state[k//(64*5)][(k//64)%5][k%64 + j] = index%2
            index = index//2    
        k-=4
        c_ind -=1
    
    current_round = rounds-1
    tempstate = np.array(empty_array).reshape(5,5,64)

    while current_round>0:
          #reverse chi
          for i in range(0,5):
            for k in range(0,64):
                t=""
                s=""
                for j in range(0,5):
                  s += str(state[i][j][k])
                t = chi_inverse[s]
                for j in range(0,5):
                  tempstate[i][j][k] = t[j]

          state = tempstate.copy()

          #reverse pi
          for i in range(5):
            for j in range(5):
                for k in range(64):
                  tempstate[i][j][k] = state[j][((2 * i) + (3 * j)) % 5][k]

          #reverse theta
          state= tempstate.copy()
          column_parity = np.zeros((5,64),dtype='int')
          for i in range(0,5):
              for k in range(64):
                  column_parity[i][k] = 0
                  #for j in range(5):
                  column_parity[i][k] ^= tempstate[i][0][k] ^ tempstate[i][1][k] ^ tempstate[i][2][k] ^ tempstate[i][3][k] ^ tempstate[i][4][k]

          for i in range(0,5):
            for k in range(64):
                #for j in range(5):
                c = column_parity[(i+2)%5][k] ^ column_parity[(i+3)%5][k]
                state[i][0][k] = tempstate[i][0][k] ^ c
                state[i][1][k] = tempstate[i][1][k] ^ c
                state[i][2][k] = tempstate[i][2][k] ^ c
                state[i][3][k] = tempstate[i][3][k] ^ c
                state[i][4][k] = tempstate[i][4][k] ^ c

          current_round -= 1


    message = np.zeros(r,dtype='int')
    for k in range(r):
		    message[k] = state[k//(64*5)][(k//64) % 5][k%64] 


    password = ""
    for i in range(0,r,8):
        tmp = [str(x) for x in message[i:i+8]]
        tmp =  chr(int("".join(tmp),2))
        if tmp.isalnum():
            password +=""+tmp
        else:
            password+=" "


    return password 

In [ ]:
print(break_sha(myhash1,rounds))
print(break_sha(myhash2,rounds))

    fybx    fybxoowtfybx    fybx    fybx            fybx    fybx    fybx
oowtfybx        ha                      iggd    iggd    nippfybxoowtfybx
